In [1]:

import pandas as pd
import numpy as np
import time
import seaborn as sb
import datetime as dt

In [2]:
t = '5/1/2020 16:15'
def format_date(t):
    if len(t)<11:
        t = t+' 12:00' #somehow the data is missing the 12pm timem stamp
    date,time = t.split(' ')
    month,day,year = date.split('/')
    hr,sec = time.split(':')

    return dt.datetime(int(year),int(month),int(day),int(hr),int(sec))
print(format_date(t).minute)

15


In [99]:
def test_strat(data,closing_hour,closing_minute,opening_hour,opening_minute):
    time_in_min = opening_hour *60 +opening_minute
    print(time_in_min)
    win = 0 
    lose = 0
    win_list = []
    lose_list = []
    current_day = None
    minute_counter = 0 
    price_weight = 0
    volume  = 0 
    close_price = -1
    VMAP = 0 
    side = -1 #1 indicates buy and 0 indicates sell
    entry_price =0 

    for index, row in data.iterrows():
        date = format_date(row['Date'])
        if date.hour>15:
            continue

        if date.hour<9: 
            continue

        if date.hour == 9 and date.minute<30:
            continue

        if current_day == None or date.day != current_day: # new data reset everyhing
            #new_day
            current_day = date.day
            minute_counter  = 0
            price_weight = 0
            volume = 0
            if close_price == 0:
                print("missing closing for the day before" +str(date))
            close_price = 0
            VMAP = 0
            entry_price =0
            side = -1

        volume+=row['Volume']
        price_weight+=row['Volume']*(row['High']+row['Low'])/2
        
        VMAP = price_weight/volume
        if (date.hour*60 + date.minute) >=time_in_min:
            if row['Open']>VMAP and entry_price == 0:
                side =1
            elif row['Open']<=VMAP and entry_price == 0:
                side =0 
        else:
            assert(side ==-1)
                
        if side != -1:
            if side and entry_price  == 0 :#buy
                if row['Low']<VMAP: # if low less than VMAP buy
                    entry_price = VMAP

            elif side == 0 and entry_price == 0 :
                if row['High']>VMAP: #if high less than VMAP sell
                    entry_price = VMAP

        if date.hour ==closing_hour and date.minute ==closing_minute:
            close_price = row['Open']
            
            if entry_price !=0 and close_price != 0:
                if side:#buy
                    if entry_price<close_price:
                        win +=1
                        win_list.append((side,(close_price-entry_price)/entry_price,entry_price,close_price))
                    else:
                        lose+=1
                        lose_list.append((side,(close_price-entry_price)/entry_price,entry_price,close_price))

                else:
                    if entry_price>close_price:
                        win +=1
                        win_list.append((side,(entry_price-close_price)/entry_price,entry_price,close_price))
                    else:
                        lose+=1
                        lose_list.append((side,(entry_price-close_price)/entry_price,entry_price,close_price))

    return win_list,lose_list

In [100]:
#sanity check should be  a 3 no trades while last one should hav one winning trade
days =['2/11/2020', '2/25/2020','2/26/2020','4/21/2020']
data = pd.read_csv('SPY.csv')
for day in days:
    test_data = data.Date.str.contains(day)

    win_list,lose_list =    test_strat(data[test_data],15,59,15,30)
    total_win = 0
    total_lose =0 
    for i in win_list:
        total_win +=i[1]
    for i in lose_list:
        total_lose +=i[1]
    print(win_list)
    print(lose_list)
    print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(total_win)))
    print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(total_lose))) 

930
[]
[]
total of 0 wining trades at a 0 gain
total of 0 losing trades at a 0 loss
930
[]
[]
total of 0 wining trades at a 0 gain
total of 0 losing trades at a 0 loss
930
[]
[]
total of 0 wining trades at a 0 gain
total of 0 losing trades at a 0 loss
930
[(0, 0.0033385551455344773, 274.556622424536, 273.64)]
[]
total of 1 wining trades at a 0.0033385551455344773 gain
total of 0 losing trades at a 0 loss


In [90]:
files = ['TLT.csv','SPY.csv','APPL.CSV', 'IWM.csv' ,'AMZN.csv']
for filename in files:
    data = pd.read_csv(filename)
    print(filename)
    win_list,lose_list =test_strat(data,15,59,15,30)
    total_win = 0
    total_lose =0 
    for i in win_list:
        total_win +=i[1]
    for i in lose_list:
        total_lose +=i[1]
        
    print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(total_win)))
    print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(total_lose)))

TLT.csv
930
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 25 wining trades at a 0.05038925720274313 gain
total of 18 losing trades at a -0.012196054179025652 loss
SPY.csv
930
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 27 wining trades at a 0.09916794190588231 gain
total of 27 losing trades at a -0.07165358635360376 loss
APPL.CSV
930
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 24 wining trades at a 0.23190987509829783 gain
total of 21 losing trades at a -0.14311038251994734 loss
IWM.csv
930
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 29 wining trades at a 0.08747264714089599 gain
total of 28 losing trades at a -0.06985977166304978 loss
AMZN.csv
930
missing closing for the day before2019-12-02 09:30:00

In [91]:
#marked at 349 instead of 359
files = ['TLT.csv','SPY.csv','APPL.CSV', 'IWM.csv' ,'AMZN.csv']
for filename in files:
    data = pd.read_csv(filename)
    print(filename)
    win_list,lose_list =test_strat(data,15,49,15,30)
    total_win = 0
    total_lose =0 
    for i in win_list:
        total_win +=i[1]
    for i in lose_list:
        total_lose +=i[1]
        
    print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(total_win)))
    print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(total_lose)))

TLT.csv
930
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 21 wining trades at a 0.026525370507261246 gain
total of 12 losing trades at a -0.014140599861423502 loss
SPY.csv
930
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 24 wining trades at a 0.04880389366361768 gain
total of 17 losing trades at a -0.014543765908779057 loss
APPL.CSV
930
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 26 wining trades at a 0.08525502794344565 gain
total of 13 losing trades at a -0.036850134197698246 loss
IWM.csv
930
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 24 wining trades at a 0.04662371734739958 gain
total of 13 losing trades at a -0.01607428803046751 loss
AMZN.csv
930
missing closing for the day before2019-12-02 09:30

In [93]:
# 2 hour window instead of 30 mins
files = ['TLT.csv','SPY.csv','APPL.CSV', 'IWM.csv' ,'AMZN.csv']
for filename in files:
    data = pd.read_csv(filename)
    print(filename)
    win_list,lose_list =test_strat(data,15,59,14,0)
    total_win = 0
    total_lose =0 
    for i in win_list:
        total_win +=i[1]
    for i in lose_list:
        total_lose +=i[1]
        
    print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(total_win)))
    print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(total_lose)))

TLT.csv
840
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 45 wining trades at a 0.12501508314658594 gain
total of 39 losing trades at a -0.13532783993905065 loss
SPY.csv
840
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 46 wining trades at a 0.25764287694766286 gain
total of 48 losing trades at a -0.22825323529687377 loss
APPL.CSV
840
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 32 wining trades at a 0.31897271808080696 gain
total of 40 losing trades at a -0.28061503123912634 loss
IWM.csv
840
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 36 wining trades at a 0.2127203461553225 gain
total of 47 losing trades at a -0.2535561255227492 loss
AMZN.csv
840
missing closing for the day before2019-12-02 09:30:00
mi

In [94]:
def test_strat_prev_day(data):
    win = 0 
    lose = 0
    win_list = []
    lose_list = []
    win_list_up = []
    lose_list_up = []
    win_list_down = []
    lose_list_down = []
    current_day = None
    minute_counter = 0 
    price_weight = 0
    volume  = 0 
    close_price = -1
    VMAP = 0 
    side = -1 #1 indicates buy and 0 indicates sell
    entry_price = 0 
    prev_closing = 0
    allowed_side = -1
    for index, row in data.iterrows():
        date = format_date(row['Date'])
        if date.hour>15:
            continue

        if date.hour<9: 
            continue

        if date.hour == 9 and date.minute<30:
            continue

        if current_day == None or date.day != current_day: # new data reset everyhing
            #new_day
            current_day = date.day
            minute_counter  = 0
            price_weight = 0
            volume = 0
            if close_price == 0:
                print("missing closing for the day before" +str(date))
            close_price = 0
            VMAP = 0
            entry_price =0
            side = -1
            if row['Open']>=prev_closing*1.002:
                allowed_side = 1
            elif row['Open']<=prev_closing*0.998:
                allowed_side =0
            else:
                allowed_side = -1 
            if prev_closing == 0:
                allowed_side = -1

                
        volume+=row['Volume']
        price_weight+=row['Volume']*(row['High']+row['Low'])/2
        
        VMAP = price_weight/volume#this might be a problem since there feed forward of data
        if date.hour>=14 and date.minute>=0:
            if row['Open']>VMAP and entry_price == 0:
                side =1
            elif row['Open']<=VMAP and entry_price == 0:
                side =0 

        if side != -1 and (allowed_side == -1 or allowed_side == side):
            if side and entry_price  == 0 :#buy
                if row['Low']<VMAP: # if low less than VMAP buy
                    entry_price = VMAP
            elif side == 0 and entry_price ==0:
                if row['High']>VMAP: #if high less than VMAP sell
                    entry_price = VMAP

        if date.hour ==15 and date.minute ==59:
            close_price = row['Open']
            prev_closing = row['Close']
            
            if entry_price !=0 and close_price != 0:
  
                if side:#buy
                    if entry_price<close_price and allowed_side ==-1:
                        win +=1
                        win_list.append((side,(close_price-entry_price)/entry_price))
                    elif entry_price>=close_price and allowed_side ==-1:
                        lose+=1
                        lose_list.append((side,(close_price-entry_price)/entry_price))
                    elif entry_price<close_price and allowed_side ==1:
                        win +=1
                        win_list_up.append((side,(close_price-entry_price)/entry_price))
                    elif entry_price>=close_price and allowed_side ==1:
                        lose+=1
                        lose_list_up.append((side,(close_price-entry_price)/entry_price))
                    else:
                        raise Exception
                else:
                    if entry_price>close_price and allowed_side ==-1:
                        win +=1
                        win_list.append((side,(entry_price-close_price)/entry_price))
                    elif entry_price<=close_price and allowed_side==-1:
                        lose+=1
                        lose_list.append((side,(entry_price-close_price)/entry_price))
                    elif entry_price>close_price and allowed_side ==0:
                        win +=1
                        win_list_down.append((side,(entry_price-close_price)/entry_price))
                    elif entry_price<=close_price and allowed_side==0:
                        lose+=1
                        lose_list_down.append((side,(entry_price-close_price)/entry_price))
                    else:
                        raise Exception

    return win_list,lose_list, win_list_up, lose_list_up,win_list_down ,lose_list_down

In [95]:
#up day and down day
files = ['TLT.csv','SPY.csv','APPL.CSV', 'IWM.csv' ,'AMZN.csv']
for filename in files:
    data = pd.read_csv(filename)
    print(filename)
    win_list,lose_list, win_list_up, lose_list_up,win_list_down ,lose_list_down =test_strat_prev_day(data)
    total_win = 0
    total_lose =0 
    for i in win_list:
        total_win +=i[1]
    for i in lose_list:
        total_lose +=i[1]
        
    print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(sum(item[1] for item in win_list))))
    print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(sum(item[1] for item in lose_list))))
        
    print("total of {} wining trades on up days at a {} gain".format(str(len(win_list_up)),str(sum(item[1] for item in win_list_up))))
    print("total of {} losing trades on up days at a {} loss".format(str(len(lose_list_up)),str(sum(item[1] for item in lose_list_up))))

    
    print("total of {} wining trades on down days at a {} gain".format(str(len(win_list_down)),str(sum(item[1] for item in win_list_down))))
    print("total of {} losing trades on down days at a {} loss".format(str(len(lose_list_down)),str(sum(item[1] for item in lose_list_down))))
    

TLT.csv
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 11 wining trades at a 0.04348446796493509 gain
total of 4 losing trades at a -0.0041107385905999605 loss
total of 20 wining trades on up days at a 0.07956109771567418 gain
total of 21 losing trades on up days at a -0.08519343045595051 loss
total of 10 wining trades on down days at a 0.024003073192898343 gain
total of 16 losing trades on down days at a -0.018271455262310297 loss
SPY.csv
missing closing for the day before2019-12-02 09:30:00
missing closing for the day before2019-12-26 09:30:00
total of 11 wining trades at a 0.015369795454333448 gain
total of 16 losing trades at a -0.026399592059956147 loss
total of 19 wining trades on up days at a 0.20977031432830393 gain
total of 16 losing trades on up days at a -0.042408819148017196 loss
total of 18 wining trades on down days at a 0.11019828068769784 gain
total of 12 losing trades on down days at a -0.06658355164

In [96]:
#test case 21 is a down day should have a sell trade for a gain.
day1 = '4/20/2020'
day2 ='4/21/2020'
data = pd.read_csv('SPY.csv')
test_data = data.Date.str.contains(day1)
test_data2= data.Date.str.contains(day2)
data_day1=data[test_data]
data_day2=data[test_data2]
vertical_stack = pd.concat([data_day1,data_day2], axis=0)


win_list,lose_list, win_list_up, lose_list_up,win_list_down ,lose_list_down =test_strat_prev_day(vertical_stack)


print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(sum(item[1] for item in win_list))))
print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(sum(item[1] for item in lose_list))))

print("total of {} wining trades on up days at a {} gain".format(str(len(win_list_up)),str(sum(item[1] for item in win_list_up))))
print("total of {} losing trades on up days at a {} loss".format(str(len(lose_list_up)),str(sum(item[1] for item in lose_list_up))))


print("total of {} wining trades on down days at a {} gain".format(str(len(win_list_down)),str(sum(item[1] for item in win_list_down))))
print("total of {} losing trades on down days at a {} loss".format(str(len(lose_list_down)),str(sum(item[1] for item in lose_list_down))))


#test case
#test case 28 is a up day should have one losing trade
day1 = '4/27/2020'
day2 ='4/28/2020'
data = pd.read_csv('SPY.csv')
test_data = data.Date.str.contains(day1)
test_data2= data.Date.str.contains(day2)
data_day1=data[test_data]
data_day2=data[test_data2]
vertical_stack = pd.concat([data_day1,data_day2], axis=0)


win_list,lose_list, win_list_up, lose_list_up,win_list_down ,lose_list_down =test_strat_prev_day(vertical_stack)


print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(sum(item[1] for item in win_list))))
print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(sum(item[1] for item in lose_list))))

print("total of {} wining trades on up days at a {} gain".format(str(len(win_list_up)),str(sum(item[1] for item in win_list_up))))
print("total of {} losing trades on up days at a {} loss".format(str(len(lose_list_up)),str(sum(item[1] for item in lose_list_up))))


print("total of {} wining trades on down days at a {} gain".format(str(len(win_list_down)),str(sum(item[1] for item in win_list_down))))
print("total of {} losing trades on down days at a {} loss".format(str(len(lose_list_down)),str(sum(item[1] for item in lose_list_down))))


#test case

total of 0 wining trades at a 0 gain
total of 1 losing trades at a -0.008083604651736792 loss
total of 0 wining trades on up days at a 0 gain
total of 0 losing trades on up days at a 0 loss
total of 1 wining trades on down days at a 0.0037430125838138436 gain
total of 0 losing trades on down days at a 0 loss
total of 0 wining trades at a 0 gain
total of 0 losing trades at a 0 loss
total of 0 wining trades on up days at a 0 gain
total of 1 losing trades on up days at a -0.007763309734921721 loss
total of 0 wining trades on down days at a 0 gain
total of 0 losing trades on down days at a 0 loss


In [143]:
def test_strat_VMAP(data,closing_hour,closing_minute,opening_hour,opening_minute):
    time_in_min = opening_hour *60 +opening_minute
    win = 0 
    lose = 0
    win_list = []
    lose_list = []
    current_day = None
    minute_counter = 0 
    price_weight = 0
    volume  = 0 
    close_price = -1
    VMAP = 0 
    side = -1 #1 indicates buy and 0 indicates sell
    entry_price =0 
    prev_VMAP = 0
    for index, row in data.iterrows():
        date = format_date(row['Date'])
        if date.hour>15:
            continue

        if date.hour<9: 
            continue

        if date.hour == 9 and date.minute<30:
            continue

        if current_day == None or date.day != current_day: # new data reset everyhing
            #new_day
            current_day = date.day
            minute_counter  = 0
            price_weight = 0
            volume = 0
            if close_price == 0:
                print("missing closing for the day before" +str(date))
            close_price = 0
            prev_VMAP = VMAP
            VMAP = 0
            entry_price =0
            side = -1
        
        volume+=row['Volume']
        price_weight+=row['Volume']*(row['High']+row['Low'])/2
        
        VMAP = price_weight/volume
        if prev_VMAP == 0:
            continue
        if (date.hour*60 + date.minute) >=time_in_min:

            if row['Open']>prev_VMAP and entry_price == 0:
                side =1
            elif row['Open']<=prev_VMAP and entry_price == 0:
                side =0 
        else:
            assert(side ==-1)
                
        if side != -1:

            if side and entry_price  == 0 :#buy

                if row['Low']<prev_VMAP: # if low less than VMAP buy
                    entry_price = prev_VMAP

            elif side == 0 and entry_price == 0 :
                if row['High']>prev_VMAP: #if high less than VMAP sell
                    entry_price = prev_VMAP

        if date.hour ==closing_hour and date.minute ==closing_minute:
            close_price = row['Open']
            
            if entry_price !=0 and close_price != 0:
                if side:#buy
                    if entry_price<close_price:
                        win +=1
                        win_list.append((side,(close_price-entry_price)/entry_price))
                    else:
                        lose+=1
                        lose_list.append((side,(close_price-entry_price)/entry_price))

                else:
                    if entry_price>close_price:
                        win +=1
                        win_list.append((side,(entry_price-close_price)/entry_price))
                    else:
                        lose+=1
                        lose_list.append((side,(entry_price-close_price)/entry_price))

    return win_list,lose_list

In [144]:
#test case 23 is higher than but falls below 22 for awhile.
day1 = '4/22/2020'
day2 ='4/23/2020'
data = pd.read_csv('SPY.csv')
test_data = data.Date.str.contains(day1)
test_data2= data.Date.str.contains(day2)
data_day1=data[test_data]
data_day2=data[test_data2]
data = pd.concat([data_day1,data_day2], axis=0)

win_list,lose_list =test_strat_VMAP(data,15,59,9,30)
total_win = 0
total_lose =0 
for i in win_list:
    total_win +=i[1]
for i in lose_list:
    total_lose +=i[1]

print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(total_win)))
print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(total_lose)))


missing closing for the day before2020-04-23 09:30:00
total of 1 wining trades at a 0.001598394066443215 gain
total of 0 losing trades at a 0 loss


In [145]:
# 2 hour window instead of 30 mins
files = ['TLT.csv','SPY.csv','APPL.CSV', 'IWM.csv' ,'AMZN.csv']
for filename in files:
    data = pd.read_csv(filename)
    print(filename)
    win_list,lose_list =test_strat_VMAP(data,10,0,9,30)
    total_win = 0
    total_lose =0 
    for i in win_list:
        total_win +=i[1]
    for i in lose_list:
        total_lose +=i[1]
        
    print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(total_win)))
    print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(total_lose)))

TLT.csv
missing closing for the day before2019-10-22 09:30:00
total of 7 wining trades at a 0.01747879208247665 gain
total of 10 losing trades at a -0.018817322476958335 loss
SPY.csv
missing closing for the day before2019-10-22 09:30:00
total of 23 wining trades at a 0.090619572452466 gain
total of 19 losing trades at a -0.02620995272582589 loss
APPL.CSV
missing closing for the day before2019-10-22 09:30:00
total of 36 wining trades at a 0.166826477208257 gain
total of 16 losing trades at a -0.04051427928642086 loss
IWM.csv
missing closing for the day before2019-10-22 09:30:00
total of 25 wining trades at a 0.1584997020483116 gain
total of 22 losing trades at a -0.09152788535598623 loss
AMZN.csv
missing closing for the day before2019-10-22 09:30:00
total of 28 wining trades at a 0.11281234003613319 gain
total of 24 losing trades at a -0.09510463798936764 loss
